In [1]:
import carla
import math
import random
import time

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()
world_map = world.get_map()

In [3]:
# Get the blueprint of a black car 
blueprint_library = world.get_blueprint_library()
vehicle_bp = random.choice(blueprint_library.filter('vehicle.toyota.prius'))
vehicle_bp.set_attribute('color', '0,0,0')

In [4]:
# Spawn Prius at given location
spawn_point = carla.Transform(
    carla.Location(x=-58.468666, y=188.412048, z=0.06),#, 7.642653
    carla.Rotation(yaw=-90)
)
prius_bp = blueprint_library.find('vehicle.toyota.prius')
vehicle = world.spawn_actor(prius_bp, spawn_point)
vehicle.set_simulate_physics(True)  # Ensure physics is enabled

In [5]:
# Destroy existing Prius vehicles
def destroy_existing_priuses():
    actors = world.get_actors().filter('vehicle.toyota.prius')
    for actor in actors:
        actor.destroy()
    print(f"Destroyed {len(actors)} existing Prius vehicles.")

destroy_existing_priuses()

Destroyed 1 existing Prius vehicles.


In [6]:
# street light side parking spots
spawn_points_sl = [
    carla.Transform(
    carla.Location(x=-50.468666, y=185, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
    carla.Transform(
    carla.Location(x=-50.468666, y=181, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
    carla.Transform(
    carla.Location(x=-50.468666, y=173, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
    carla.Transform(
    carla.Location(x=-50.468666, y=170, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
    carla.Transform(
    carla.Location(x=-50.468666, y=163, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
    carla.Transform(
    carla.Location(x=-50.468666, y=155, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
    ]
# prius_bp = blueprint_library.find('vehicle.toyota.prius')

# Get all vehicle blueprints
vehicle_blueprints = blueprint_library.filter('vehicle.*')

# Spawn random vehicles at each spawn point
sl_parked_vehicles = []
for sp in spawn_points_sl:
    vehicle_bp = random.choice(vehicle_blueprints)  # Select a random vehicle
    vehicle = world.spawn_actor(vehicle_bp, sp)
    sl_parked_vehicles.append(vehicle)
    vehicle.set_simulate_physics(True)  # Ensure physics is enabled

RuntimeError: Spawn failed because of collision at spawn position

In [8]:
# bill board + building side parking spots
spawn_points_bb = [carla.Transform(
    carla.Location(x=-67.468666, y=167.75, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
carla.Transform(
    carla.Location(x=-67.468666, y=187, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
carla.Transform(
    carla.Location(x=-67.468666, y=172.75, z=0.2),#, 7.642653
    carla.Rotation(yaw=0)),
carla.Transform(
    carla.Location(x=-67.468666, y=177.75, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))]

# prius_bp = blueprint_library.find('vehicle.toyota.prius')
# Get all vehicle blueprints
vehicle_blueprints = blueprint_library.filter('vehicle.*')

# Spawn random vehicles at each spawn point
bb_parked_vehicles = []
for sp in spawn_points_bb:
    vehicle_bp = random.choice(vehicle_blueprints)  # Select a random vehicle
    vehicle = world.spawn_actor(vehicle_bp, sp)
    bb_parked_vehicles.append(vehicle)
    vehicle.set_simulate_physics(True)  # Ensure physics is enabled

RuntimeError: Spawn failed because of collision at spawn position

In [9]:
def custom_destroy(parked_vehicles):
    for v in parked_vehicles:
        if v.is_alive:
            v.destroy()
    print(f"Destroyed {len(parked_vehicles)} vehicles")
    parked_vehicles.clear()

custom_destroy(sl_parked_vehicles)
custom_destroy(bb_parked_vehicles)

Destroyed 0 vehicles
Destroyed 0 vehicles


In [75]:
# Spawn walkers
walker_bp = blueprint_library.filter('walker.pedestrian.*')
walker_bp = random.choice(walker_bp)
sp_center = carla.Transform(
    carla.Location(x=-58.468666, y=170, z=0.1),#, 7.642653
    carla.Rotation(yaw=0))
sp_center_right = carla.Transform(
    carla.Location(x=-58.468666, y=160, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))
sp_center_left = carla.Transform(
    carla.Location(x=-58.468666, y=188, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))
sp_upper_left = carla.Transform(
    carla.Location(x=-67.468666, y=190, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))
sp_upper_right2 = carla.Transform(
    carla.Location(x=-67.468666, y=160, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))
sp_upper_right1 = carla.Transform(
    carla.Location(x=-67.468666, y=165, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))
sp_lower_right2 = carla.Transform(
    carla.Location(x=-45.468666, y=160, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))
sp_lower_right1 = carla.Transform(
    carla.Location(x=-45.468666, y=170, z=0.1),#, 7.642653
    carla.Rotation(yaw=0))
sp_sl_junction = carla.Transform(
    carla.Location(x=-50.468666, y=168.5, z=0.2),#, 7.642653
    carla.Rotation(yaw=0))
walker = world.spawn_actor(walker_bp, sp_center)
walker.set_simulate_physics(True)


In [74]:
for walker in world.get_actors().filter('walker.pedestrian.*'):
    walker.destroy()

In [76]:
def walk_please(walker, traj):
    walker_control = carla.WalkerControl()
    stop = carla.WalkerControl(speed=0)
    for i in range(len(traj)-1):
        dir = traj[i+1] - traj[i]
        dir.z = 0
        dir = dir/max(dir.length(), 1e-6)
        walker_control.speed = 1 + random.random()
        walker_control.direction = dir
        walker_control.jump = random.random() > 0.9
        while(walker.get_location().distance(traj[i+1]) > 1):
            walker.apply_control(walker_control)
    walker.apply_control(stop)


In [65]:
stop = carla.WalkerControl(speed=0)
walker.apply_control(stop)

In [77]:
center_to_upper_left = [sp_center.location, sp_center_left.location, sp_upper_left.location]
center_to_lower_right = [sp_center.location, sp_sl_junction.location, sp_lower_right1.location, sp_lower_right2.location]
walk_please(walker, center_to_lower_right)

In [26]:
# AI walker controller
# walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
# ai_controller = world.spawn_actor(walker_controller_bp, sp_center, walker)

# ai_controller.start()
# ai_controller.go_to_location(sp_center_right.location)
# ai_controller.set_max_speed(1 + random.random())

# time.sleep(20)

# ai_controller.stop()
# ai_controller.destroy()
# walker.destroy()